In [0]:
import requests
import json
import pandas as pd
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:

# =============================================================================
# FUNCTION 1: CHECK FOR NEW FILES
# =============================================================================

def check_new_files():
    """Check GitHub for new enhanced files to process"""
    
    # Configuration
    GITHUB_USERNAME = "AlexanderHuynhKoehler"
    REPO_NAME = "bluesky-data-pipeline"
    api_base = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}"
    
    try:
        new_files_url = f"{api_base}/contents/data/enhanced/new"
        response = requests.get(new_files_url)
        
        if response.status_code == 404:
            return []
        elif response.status_code != 200:
            print(f"❌ GitHub API error: {response.status_code}")
            return []
        
        files = response.json()
        enhanced_files = [f for f in files if f['name'].startswith('enhanced_') and f['name'].endswith('.json')]
        
        return enhanced_files
        
    except Exception as e:
        print(f"❌ Error checking new files: {e}")
        return []

# =============================================================================
# FUNCTION 2: IMPORT WITH DEDUPLICATION
# =============================================================================

def import_enhanced_data():
    """FIXED: Import with matching column names"""
    
    print("🚀 IMPORTING ENHANCED DATA (SCHEMA ALIGNED)")
    print("=" * 60)
    
    # Configuration
    GITHUB_USERNAME = "AlexanderHuynhKoehler"
    REPO_NAME = "bluesky-data-pipeline"
    api_base = f"https://api.github.com/repos/{GITHUB_USERNAME}/{REPO_NAME}"
    target_table = "social_media.bluesky_enhanced_fixed_types"
    
    try:
        # Step 1: Check for new files
        new_files = check_new_files()
        
        if not new_files:
            print("✅ No new files to process")
            return 0
        
        print(f"📥 Found {len(new_files)} new files")
        
        # Step 2: Get existing post IDs for deduplication
        try:
            existing_posts_df = spark.sql(f"""
                SELECT DISTINCT post_id 
                FROM {target_table}
                WHERE post_id IS NOT NULL AND post_id != ''
            """)
            existing_post_ids = set([row['post_id'] for row in existing_posts_df.collect()])
            print(f"🔍 {len(existing_post_ids):,} existing post IDs")
        except:
            existing_post_ids = set()
            print("🔍 No existing posts")
        
        # Step 3: Download and deduplicate
        all_new_posts = []
        processed_files = []
        total_downloaded = 0
        duplicates_filtered = 0
        
        for file_info in new_files:
            file_response = requests.get(file_info['download_url'])
            if file_response.status_code != 200:
                continue
            
            enhanced_data = file_response.json()
            posts_data = enhanced_data.get('enhanced_posts', [])
            total_downloaded += len(posts_data)
            
            # Filter duplicates and add metadata
            for post in posts_data:
                post_id = post.get('post_id')
                if post_id and post_id not in existing_post_ids:
                    post['source_file'] = file_info['name']
                    
                    # 🛠️ CRITICAL FIX: Use 'reprocessed_at' to match existing table schema
                    post['reprocessed_at'] = datetime.now().isoformat()  # ✅ Match table column
                    
                    # Remove 'imported_at' if it exists to avoid conflicts
                    if 'imported_at' in post:
                        del post['imported_at']
                    
                    all_new_posts.append(post)
                    existing_post_ids.add(post_id)
                else:
                    duplicates_filtered += 1
            
            processed_files.append(file_info['name'])
        
        print(f"📊 Total downloaded: {total_downloaded}")
        print(f"🆕 New unique posts: {len(all_new_posts)}")
        print(f"🔄 Duplicates filtered: {duplicates_filtered}")
        
        if not all_new_posts:
            print("✅ No new unique posts")
            # Move files anyway
            for filename in processed_files:
                move_file_to_processed(filename, GITHUB_TOKEN, api_base)
            return 0
        
        # Step 4: Create DataFrame with proper types
        df = pd.DataFrame(all_new_posts)
        
        # Type enforcement
        float_cols = ['ml_sentiment_score', 'emotion_confidence', 'processing_speed', 'total_engagement']
        int_cols = ['text_length', 'text_cleaned_length', 'hashtag_count', 'mention_count']
        string_cols = ['text', 'text_cleaned', 'author', 'ml_sentiment_label', 'dominant_emotion']
        
        for col in float_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0.0).astype('float64')
        
        for col in int_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype('int64')
        
        for col in string_cols:
            if col in df.columns:
                df[col] = df[col].fillna('').astype(str)
        
        print(f"✅ DataFrame prepared: {len(df)} rows")
        
        # 🛠️ FIXED SCHEMA: Match your existing table EXACTLY
        table_schema = StructType([
            StructField("post_id", StringType(), True),
            StructField("text", StringType(), True),
            StructField("text_cleaned", StringType(), True),
            StructField("text_preview", StringType(), True),
            StructField("author", StringType(), True),
            StructField("author_display_name", StringType(), True),
            StructField("created_at", StringType(), True),
            StructField("hashtags", StringType(), True),
            StructField("mentions", StringType(), True),
            StructField("received_at", StringType(), True),
            StructField("ml_sentiment_label", StringType(), True),
            StructField("dominant_emotion", StringType(), True),
            StructField("ml_processed_at", StringType(), True),
            StructField("processed_by", StringType(), True),
            StructField("source_file", StringType(), True),
            StructField("reprocessed_at", StringType(), True),           # ✅ MATCH TABLE
            StructField("ml_sentiment_score", DoubleType(), True),
            StructField("emotion_confidence", DoubleType(), True),
            StructField("processing_speed", DoubleType(), True),
            StructField("total_engagement", DoubleType(), True),
            StructField("text_length", LongType(), True),
            StructField("text_cleaned_length", LongType(), True),
            StructField("hashtag_count", LongType(), True),
            StructField("mention_count", LongType(), True),
        ])
        
        # Select only columns that exist in both schema and data
        schema_columns = [field.name for field in table_schema.fields]
        available_columns = [col for col in schema_columns if col in df.columns]
        
        print(f"📋 Using {len(available_columns)} matching columns")
        
        # Create Spark DataFrame
        selected_df = df[available_columns].copy()
        spark_df = spark.createDataFrame(selected_df, schema=table_schema)
        
        print("✅ Spark DataFrame created with matching schema")
        
        # Step 5: Append to table with matching schema
        print("💾 Appending to Delta table...")
        
        (spark_df
            .write
            .mode("append")
            .option("mergeSchema", "false")  # Schema should match exactly now
            .saveAsTable(target_table)
        )
        
        print(f"✅ {len(all_new_posts):,} rows appended successfully!")
        
        # Step 6: Move files to processed
        moved_count = 0
        for filename in processed_files[:10]:  # Process first 10 files to test
            if move_file_to_processed(filename, GITHUB_TOKEN, api_base):
                moved_count += 1
            else:
                break  # Stop if file moving fails
        
        print(f"✅ {moved_count} files moved to processed")
        
        return len(all_new_posts)
        
    except Exception as e:
        print(f"❌ Import failed: {e}")
        import traceback
        traceback.print_exc()
        return 0

# =============================================================================
# FUNCTION 3: PIPELINE ORCHESTRATOR  
# =============================================================================

def run_pipeline():
    """Main pipeline function - checks for new files and imports them"""
    
    print("🎯 RUNNING COMPLETE IMPORT PIPELINE")
    print("=" * 60)
    
    target_table = "social_media.bluesky_enhanced_fixed_types"
    
    # Get starting row count
    try:
        start_count_df = spark.sql(f"SELECT COUNT(*) as total FROM {target_table}")
        start_count = start_count_df.collect()[0]['total']
        print(f"📊 Starting table size: {start_count:,} rows")
    except:
        start_count = 0
        print(f"📊 Table doesn't exist yet - will be created")
    
    # Check for new files
    new_files = check_new_files()
    
    if not new_files:
        print(f"✅ Pipeline complete - no new files found")
        print(f"📊 Table remains: {start_count:,} rows")
        return {
            'new_rows_added': 0,
            'starting_rows': start_count,
            'final_rows': start_count,
            'files_processed': 0
        }
    
    print(f"📥 Found {len(new_files)} new files:")
    for f in new_files:
        print(f"   - {f['name']}")
    
    # Run import
    new_rows_added = import_enhanced_data()
    
    # Get final row count
    try:
        final_count_df = spark.sql(f"SELECT COUNT(*) as total FROM {target_table}")
        final_count = final_count_df.collect()[0]['total']
    except:
        final_count = start_count + new_rows_added
    
    # Final summary
    print(f"\n" + "="*60)
    print(f"🎉 PIPELINE COMPLETE")
    print(f"=" * 60)
    print(f"📊 DELTA TABLE CHANGES:")
    print(f"   ├── Starting rows: {start_count:,}")
    print(f"   ├── NEW rows added: {new_rows_added:,}  ⭐⭐⭐")
    print(f"   └── Final rows: {final_count:,}")
    
    if new_rows_added > 0:
        growth_pct = (new_rows_added / start_count * 100) if start_count > 0 else 0
        print(f"📈 Table growth: +{growth_pct:.1f}%")
    
    print(f"✅ Files processed: {len(new_files)}")
    print("=" * 60)
    
    return {
        'new_rows_added': new_rows_added,
        'starting_rows': start_count,
        'final_rows': final_count,
        'files_processed': len(new_files)
    }

def move_file_to_processed(filename, github_token, api_base):
    """Helper: Move file from new to processed folder"""
    try:
        headers = {"Authorization": f"token {github_token}"}
        
        new_file_url = f"{api_base}/contents/data/enhanced/new/{filename}"
        response = requests.get(new_file_url, headers=headers)
        
        if response.status_code != 200:
            return False
        
        file_data = response.json()
        
        processed_file_url = f"{api_base}/contents/data/enhanced/processed/{filename}"
        upload_payload = {
            "message": f"Import complete: {filename}",
            "content": file_data['content'],
            "branch": "main"
        }
        
        upload_response = requests.put(processed_file_url, headers=headers, json=upload_payload)
        
        if upload_response.status_code in [200, 201]:
            delete_payload = {
                "message": f"Move to processed: {filename}",
                "sha": file_data['sha'],
                "branch": "main"
            }
            requests.delete(new_file_url, headers=headers, json=delete_payload)
            return True
        
        return False
        
    except:
        return False

# =============================================================================
# SIMPLE USAGE
# =============================================================================

run_pipeline()

🎯 RUNNING COMPLETE IMPORT PIPELINE
📊 Starting table size: 251 rows
📥 Found 405 new files:
   - enhanced_20250806_113101.json
   - enhanced_20250806_113103.json
   - enhanced_20250806_113105.json
   - enhanced_20250806_113108.json
   - enhanced_20250806_113110.json
   - enhanced_20250806_113113.json
   - enhanced_20250806_113115.json
   - enhanced_20250806_113118.json
   - enhanced_20250806_113120.json
   - enhanced_20250806_113123.json
   - enhanced_20250806_113125.json
   - enhanced_20250806_113127.json
   - enhanced_20250806_113130.json
   - enhanced_20250806_113132.json
   - enhanced_20250806_113135.json
   - enhanced_20250806_113137.json
   - enhanced_20250806_113140.json
   - enhanced_20250806_113142.json
   - enhanced_20250806_113144.json
   - enhanced_20250806_113147.json
   - enhanced_20250806_113149.json
   - enhanced_20250806_113151.json
   - enhanced_20250806_113154.json
   - enhanced_20250806_113156.json
   - enhanced_20250806_113159.json
   - enhanced_20250806_113201.json


{'new_rows_added': 6555,
 'starting_rows': 251,
 'final_rows': 6806,
 'files_processed': 405}